In [12]:
import pandas as pd
data = pd.read_csv("credit_card_churn.csv")

data

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.999910
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.999940
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.999980
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.999870
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.999980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,772366833,Existing Customer,50,M,2,Graduate,Single,$40K - $60K,Blue,40,...,4003.0,1851,2152.0,0.703,15476,117,0.857,0.462,0.000191,0.999810
10123,710638233,Attrited Customer,41,M,2,Unknown,Divorced,$40K - $60K,Blue,25,...,4277.0,2186,2091.0,0.804,8764,69,0.683,0.511,0.995270,0.004729
10124,716506083,Attrited Customer,44,F,1,High School,Married,Less than $40K,Blue,36,...,5409.0,0,5409.0,0.819,10291,60,0.818,0.000,0.997880,0.002118
10125,717406983,Attrited Customer,30,M,2,Graduate,Unknown,$40K - $60K,Blue,36,...,5281.0,0,5281.0,0.535,8395,62,0.722,0.000,0.996710,0.003294


In [13]:
import streamlit as st
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Load the customer churn dataset
data = pd.read_csv("credit_card_churn.csv")

# Select relevant features for the model
df = data[['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender', 'Dependent_count', 'Education_Level',
           'Marital_Status', 'Income_Category', 'Card_Category', 'Months_on_book', 'Total_Relationship_Count',
           'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy',
           'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
           'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2']]

# Streamlit app
st.title("신용카드 사용 고객의 이탈 확률 예측 모델")

# Sidebar for user input
st.sidebar.header("고객 정보")


def user_input_features():
    age = st.sidebar.slider('Customer_Age', min_value=int(df['Customer_Age'].min()), max_value=int(df['Customer_Age'].max()),
                            value=int(df['Customer_Age'].mean()))
    gender = st.sidebar.selectbox('Gender', options=[0, 1],
                                  format_func=lambda x: 'Male' if x == 1 else 'Female')
    dependents = st.sidebar.selectbox('Dependent_count', options = list(range(6)))

    education_level_options = {
    0: 'Uneducated',
    1: 'Graduate',
    2: 'High_school',
    3: 'College',
    4: 'Post_Graduate',
    5: 'Doctorate'
    }
    education_level = st.sidebar.selectbox('Education_level', options=list(education_level_options.keys()),
                                           format_func=lambda x: education_level_options[x])
    marital_status_options = {
    0: 'Marriged',
    1: 'Sigle',
    2: 'Divolced'
    }
    marital_status = st.sidebar.selectbox('Marital_Status', options=list(marital_status_options.keys()),
                                           format_func=lambda x: marital_status_options[x])
    income_category_options = {
    0: 'Less_than_$40K',
    1: '$40K_$60K',
    2: '$60K_$80K',
    3: '$80K_$120K',
    4: 'over_than_$120K'
    }
    income_category = st.sidebar.selectbox('Income_Category', options=list(income_category_options.keys()),
                                           format_func=lambda x: income_category_options[x])
    card_category_options = {
    0: 'Blue',
    1: 'Silver',
    2: 'Gold',
    3: 'Platinum',
    }
    card_category = st.sidebar.selectbox('Card_Category', options=list(card_category_options.keys()),
                                           format_func=lambda x: card_category_options[x])
    months_on_book = st.sidebar.slider('Months_on_book', min_value=int(df['Months_on_book'].min()),
                                  max_value=int(df['Months_on_book'].max()), value=int(df['Months_on_book'].mean()))
    total_relationship_count = st.sidebar.slider('Total_Relationship_Count', min_value=int(df['Total_Relationship_Count'].min()),
                                  max_value=int(df['Total_Relationship_Count'].max()), value=int(df['Total_Relationship_Count'].mean()))
    months_inactive_12_mon = st.sidebar.slider('Months_Inactive_12_mon', min_value=int(df['Months_Inactive_12_mon'].min()),
                                  max_value=int(df['Months_Inactive_12_mon'].max()), value=int(df['Months_Inactive_12_mon'].mean()))
    contacts_count_12_mon = st.sidebar.slider('Contacts_Count_12_mon', min_value=int(df['Contacts_Count_12_mon'].min()),
                                  max_value=int(df['Contacts_Count_12_mon'].max()), value=int(df['Contacts_Count_12_mon'].mean()))
    credit_limit = st.sidebar.slider('Credit_Limit', min_value=float(df['Credit_Limit'].min()),
                                       max_value=float(df['Credit_Limit'].max()),
                                       value=float(df['Credit_Limit'].mean()))    
    total_revolving_bal = st.sidebar.slider('Total_Revolving_Bal', min_value=int(df['Total_Revolving_Bal'].min()),
                                  max_value=int(df['Total_Revolving_Bal'].max()), value=int(df['Total_Revolving_Bal'].mean()))
    avg_open_to_buy = st.sidebar.slider('Avg_Open_To_Buy', min_value=int(df['Avg_Open_To_Buy'].min()),
                                  max_value=int(df['Avg_Open_To_Buy'].max()), value=int(df['Avg_Open_To_Buy'].mean()))
    total_amt_chng_q4_q1 = st.sidebar.slider('Total_Amt_Chng_Q4_Q1', min_value=float(df['Total_Amt_Chng_Q4_Q1'].min()),
                                       max_value=float(df['Total_Amt_Chng_Q4_Q1'].max()),
                                       value=float(df['Total_Amt_Chng_Q4_Q1'].mean()))
    total_trans_amt = st.sidebar.slider('Total_Trans_Amt', min_value=int(df['Total_Trans_Amt'].min()),
                                  max_value=int(df['Total_Trans_Amt'].max()), value=int(df['Total_Trans_Amt'].mean()))
    total_trans_ct = st.sidebar.slider('Total_Trans_Ct', min_value=int(df['Total_Trans_Ct'].min()),
                                  max_value=int(df['Total_Trans_Ct'].max()), value=int(df['Total_Trans_Ct'].mean()))
    total_ct_chng_q4_q1 = st.sidebar.slider('Total_Ct_Chng_Q4_Q1', min_value=float(df['Total_Ct_Chng_Q4_Q1'].min()),
                                       max_value=float(df['Total_Ct_Chng_Q4_Q1'].max()),
                                       value=float(df['Total_Ct_Chng_Q4_Q1'].mean()))
    avg_utilization_ratio = st.sidebar.slider('Avg_Utilization_Ratio', min_value=float(df['Avg_Utilization_Ratio'].min()),
                                       max_value=float(df['Avg_Utilization_Ratio'].max()),
                                       value=float(df['Avg_Utilization_Ratio'].mean()))

    data = {
        'Customer_Age': customer_age,
        'Gender': gender,
        'Dependent_count': dependent_count,
        'Education_Level': education_level,
        'Marital_Status': marital_status,
        'Income_Category': income_category,
        'Card_Category': card_category,
        'Months_on_book': months_on_book,
        'Total_Relationship_Count': total_relationship_count,
        'Months_Inactive_12_mon': months_inactive_12_mon,
        'Contacts_Count_12_mon': contacts_count_12_mon,
        'Credit_Limit': credit_limit,
        'Total_Revolving_Bal': total_revolving_bal,
        'Avg_Open_To_Buy': avg_open_to_buy,
        'Total_Amt_Chng_Q4_Q1': total_amt_chng_q4_q1,
        'Total_Trans_Amt': total_trans_amt,
        'Total_Trans_Ct': total_trans_ct,
        'Total_Ct_Chng_Q4_Q1': total_ct_chng_q4_q1,
        'Avg_Utilization_Ratio': avg_utilization_ratio
    }
    features = pd.DataFrame(data, index=[0])
    return features

input_df = user_input_features()

# Display user input
st.subheader("고객 정보를 입력하세요")
st.write(input_df)

# 모델 훈련 과정이 들어갈 자리입니다
# # Splitting data for training and testing
# X = df.drop(columns=['Churn'])
# y = df['Churn']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Training
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# model = LogisticRegression()
# model.fit(X_train_scaled, y_train)

# # Prediction
# input_scaled = scaler.transform(input_df)
# prediction = model.predict(input_scaled)
# prediction_proba = model.predict_proba(input_scaled)

# # Display Prediction
# st.subheader("Prediction")
# if prediction[0] == 1:
#     st.write("This customer is likely to churn.")
# else:
#     st.write("This customer is not likely to churn.")

# # Display Prediction Probability
# st.subheader("Prediction Probability")
# st.write(f"Probability of Not Churning: {prediction_proba[0][0]:.2f}")
# st.write(f"Probability of Churning: {prediction_proba[0][1]:.2f}")

# # Model Accuracy
# st.subheader("Model Accuracy")
# y_pred = model.predict(X_test_scaled)
# st.write(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")

2024-11-13 17:34:35.942 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 17:34:35.942 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 17:34:35.943 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 17:34:35.944 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 17:34:35.946 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 17:34:35.946 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 17:34:35.947 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 17:34:35.947 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

NameError: name 'customer_age' is not defined